In [8]:
import nltk, re, pprint
from nltk import word_tokenize
import os,json
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize
import augur
import sqlalchemy as s
import pandas as pd
%matplotlib inline
#nltk.download('punkt')
#nltk.download('stopwords')
#pip install twython
#nltk.download('vader_lexicon')

In [9]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()

In [10]:
table_names = s.inspect(connect.db).get_table_names()
print(table_names)

['issue_response_time', 'mail_lists', 'mailing_list_jobs']


In [11]:
if("mailing_list_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    print(df1)
    val = True

          project
0      aalldp-dev
1  archetypes-dev


In [51]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sid = SentimentIntensityAnalyzer()
for i in df1['project'].values:
    print(i,"Mailing List")
    SQL = s.sql.text("""SELECT subject,message_id,message_text 
    FROM mail_lists WHERE mailing_list = """ +  "'" + i + "'")
    df2 = pd.read_sql(SQL, connect.db)
    #print(df2)
    grouped = df2.groupby('message_id').groups
    print("Here!!!!")
    #print("\n\n","-"*70,"\n\n")
    for group in grouped:
        message = (df2.loc[df2['message_id'] == group]['message_text']).tolist()
        message_text = ''.join(message)
        sentences = tokenizer.tokenize(message_text)
        for sentence in sentences:
            scores = sid.polarity_scores(sentence)
            #print(scores)
            print(sentence)
            for key in sorted(scores):
                    print('{0}: {1}, '.format(key, scores[key]), end='')
        print("\n\n")
        print("\n\n\n")
    print("\n\n")

aalldp-dev Mailing List
Here!!!!
Hi AALLDP Team,



1.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, Does your project have any plans to be archived?
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, If your project has no plans for any future active development, project committers can vote to move a project to the Archived State by a Termination Review according to the OpenDaylight Project Lifecycle and Releases document [1].
compound: 0.128, neg: 0.058, neu: 0.871, pos: 0.071, 2.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, If you projects plans to remain active, does it intend to participate in the Boron Simultaneous Release as documented in the Boron Release Plan [2]?
compound: 0.4019, neg: 0.0, neu: 0.899, pos: 0.101, If not, could someone please publicly decline to participate with a message like so: "The <PROJECT_NAME> project is formally declining to participate in the Boron SR".
compound: 0.1376, neg: 0.071, neu: 0.838, pos: 0.091, There are no project related activity on Gerrit.
co